In [1]:
from astroquery.vizier import Vizier
import numpy as np
import pyds9
from glob import glob
from astropy.io import fits
from pyds9 import *
from astropy import wcs
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
from regions import read_ds9, write_ds9
from regions import DS9Parser
from regions import ds9_objects_to_string
from astropy.coordinates import Angle, SkyCoord
from regions import CircleSkyRegion
from astroquery.simbad import Simbad
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord

# This notebook will detect sources from a vizier catalogue  in a give file(s) using ds9.

<b> This notebook must be run off the NRAO server <b>

In [9]:
def pipeline(files,catalog='VIII/100/gleamegc'):
    #Vizier.ROW_LIMIT = -1
    #table = Vizier.get_catalogs(catalog='VIII/100/gleamegc')[0]
    #Gleam_ra = table['RAJ2000']
    #Gleam_dec = table['DEJ2000']
    #Gleam_flux = table['Fintwide']
    
    
   #Simbad query each source and make list of ra and dec
   #ra = []
   #dec = []
   #new_sources = []


   #for sources in source_list:
   #    #print(sources)
   #    result_table = Simbad.query_object(sources)
   #    if result_table == None:
   #        pass
   #    else:

   #        ra.append(result_table['RA'])
   #        dec.append(result_table['DEC'])
   #        new_sources.append(sources)
   #
    
    #define a ra/dec mask so only sources in mosaics are chosen
    #I should edit this, seems like cuts arent very good...
    deccuts , racuts, fluxmin = [-41,-21] , [0,161] , 3 #deg,deg,Jy
    #create a mask for ra,dec and flux
    w = []
    #for i in range(len(Gleam_dec)):
    #    w.append((deccuts[0] < Gleam_dec[i] < deccuts[1]) & (racuts[0] < Gleam_ra[i] < racuts[1]) & (Gleam_flux[i] > fluxmin))
 
    # #sources that fit our criteria
   # data = table[w]
   # Gleam_ra = data['RAJ2000']
   # Gleam_dec = data['DEJ2000']
   # Gleam_flux = data['Fintwide']
   # np.savez('gleam_sources',Gleam_ra,Gleam_dec)
    
    # #open an instance of ds9
    ds9 = pyds9.DS9()
    
    #lets read in exoplanet catalog?
    filename = '/home/gonzalo/Downloads/exoplanets.csv'
    data = pd.read_csv(filename,engine='python')
    ra = data['ra_str']
    dec = data['dec_str']
    #for i in range(len(files)):
    print(files + ' .... done')
    #opens wcs solutions in each header
    hdu = fits.open(files,mode='readonly', ignore_missing_end=True)
    imwcs = wcs.WCS(hdu[0].header)


    #loads each file into ds9 and circles GLEAM SOURCES
    ds9.set_pyfits(hdu)
    #ds9.set('frame {}'.format(i+1))
    ds9.set('file {}'.format(files))
    ds9.set('scale zscale')
    ds9.set('zoom to fit')

    # Applies wcs solutions to translate GLEAM positions to pixel values of fits image
    #refx,refy = imwcs.wcs_world2pix(Gleam_ra[:], Gleam_dec[:],0)
    #print(ra,dec)
    refx = []
    refy = []
    for i,j in zip(ra,dec):
        c = SkyCoord(i,j, frame='icrs')
        refx.append(c.ra.value)
        refy.append(c.dec.value)
    
    #for i in range(len(ra)):
    x,y = imwcs.wcs_world2pix(refx,refy,0)

    
    uv_ceti = Simbad.query_object('UV Ceti')
    ra,dec = uv_ceti['RA'],uv_ceti['DEC']
    c = SkyCoord(ra[0],dec[0], frame='icrs',unit=(u.hourangle, u.deg))
    ra = c.ra.value
    dec = c.dec.value
    
    ra,dec = imwcs.wcs_world2pix(ra,dec,0)
    for i in range(len(refx)):
        ds9.set('regions', 'image; circle {},{} {} #color = green'.format(x[i],y[i], 15))
        #print(c.ra.value)
        ds9.set('regions', 'image; circle {},{} {} #color = red'.format(ra,dec, 15))



        #ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))
        #ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))
        #ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))
        #ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))
        #ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))
        #ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))


    
    #for i in range(len(Gleam_dec)):
    #    ds9.set('regions', 'image; circle {},{} {} #color = green'.format(refx[i],refy[i], 15))

    return print('Success! Matched a total of {} sources!'.format(len(refx))) 

    

In [10]:
%%time
#here I specify the mosaiced fits files i want to use for the "pipeline"
#files = sorted(glob('masterpmaps/*.fits'))ra
files = '/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorVnewd.fits'
pipeline(files)

/home/gonzalo/Desktop/polarizedmastermaps/mastermapbeamcorVnewd.fits .... done
Success! Matched a total of 4093 sources!
CPU times: user 7.79 s, sys: 22.2 s, total: 29.9 s
Wall time: 1min 8s
